In [2]:
import csv
import codecs
from tqdm import tqdm
from urllib.request import urlretrieve
import json

In [3]:
lines = None
with open('../data/GO/go.obo') as f:
    lines = f.readlines()

In [4]:
cnt = 0
go_dic = {}
for line in lines:
    # if cnt == 2:
    #     break
    line.strip()
    if line.find('id:') == 0:
        go = line[line.find('GO:'):].strip()
        # print(go)
        # cnt += 1
    elif line.find('namespace:') == 0:
        namespace = line[10:].strip()
        # print(namespace)
        # cnt += 1
        go_dic[go] = namespace

    

In [5]:
cnt = 0
go_name_dic = {}
for line in lines:
    # if cnt == 2:
    #     break
    line.strip()
    if line.find('id:') == 0:
        go = line[line.find('GO:'):].strip()
        # print(go)
        # cnt += 1
    elif line.find('name:') == 0:
        name = line[5:].strip()
        # print(namespace)
        # cnt += 1
        go_name_dic[go] = name


In [6]:
len(go_name_dic)

47596

In [7]:
go_dic['GO:0050026']

'molecular_function'

In [8]:
go_rows = []
with codecs.open('../data/GO/pdb_chain_go.csv', encoding='utf-8') as f:
    for go_row in csv.DictReader(f, skipinitialspace=True):
        go_rows.append(go_row)
print(len(go_rows))

8650440


In [ ]:
go_rows[0]

{'PDB': '101m',
 'CHAIN': 'A',
 'SP_PRIMARY': 'P02185',
 'WITH_STRING': 'UniProtKB-KW:KW-0479',
 'EVIDENCE': 'IEA',
 'GO_ID': 'GO:0046872'}

In [9]:
pdb_chain_go_lines = None
with open('../data/GO/pdb_chain_go.csv', encoding='utf-8') as f:
    pdb_chain_go_lines = f.readlines()
cnt = 0
pdb_chain_go_rows = []
for line in tqdm(pdb_chain_go_lines):
    cnt += 1
    if cnt == 1:
        continue
    elements = line.split(',')
    row = {}
    row['PDB'] = elements[0]
    row['CHAIN'] = elements[1]
    row['uniprot'] = elements[2]
    row['GO_ID'] = elements[-1].strip()
    row['EVIDENCE'] = elements[-2]
    pdb_chain_go_rows.append(row)
print(cnt)

100%|██████████| 8650441/8650441 [00:31<00:00, 272211.72it/s]

8650441


In [ ]:
pdb_chain_go_rows[1]

{'PDB': '101m',
 'CHAIN': 'A',
 'uniprot': 'P02185',
 'GO_ID': 'GO:0005344',
 'EVIDENCE': 'IEA'}

In [ ]:
go_rows[0]

{'PDB': '101m',
 'CHAIN': 'A',
 'SP_PRIMARY': 'P02185',
 'WITH_STRING': 'UniProtKB-KW:KW-0479',
 'EVIDENCE': 'IEA',
 'GO_ID': 'GO:0046872'}

In [ ]:
from tqdm import tqdm
go_annot = {}
for row in tqdm(pdb_chain_go_rows):
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    # pdb_chain = pdb_chain.upper()
    # print(row['GO_ID'])
    try:
        if pdb_chain in go_annot.keys():
            go_annot[pdb_chain][go_dic[row['GO_ID']]].append(row['GO_ID'])
        else:
            go_annot[pdb_chain] = {}
            go_annot[pdb_chain]['uniprot'] = row['uniprot']
            go_annot[pdb_chain]['molecular_function'] = []
            go_annot[pdb_chain]['biological_process'] = []
            go_annot[pdb_chain]['cellular_component'] = []
            go_annot[pdb_chain][go_dic[row['GO_ID']]].append(row['GO_ID'])
    except Exception as e:
        print(e, pdb_chain, row['GO_ID'])

  0%|          | 31856/8650440 [00:00<00:27, 318536.76it/s]

100%|██████████| 8650440/8650440 [00:36<00:00, 239433.83it/s]


In [ ]:
print(go_annot['1SZ7-A'])
print(len(go_annot))

{'uniprot': 'O43617', 'molecular_function': ['GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515', 'GO:0005515'], 'biological_process': ['GO:0006888', 'GO:0006891', 'GO:0016192', 'GO:0006888', 'GO:0006888', 'GO:0006901', 'GO:0048208', 'GO:0099022'], 'cellular_component': ['GO:0030008', 'GO:0005829', 'GO:0033106', 'GO:0005783', 'GO:0005794', 'GO:0005783', 'GO:0005794', 'GO:0000139', 'GO:0005829', 'GO:0030008', 'GO:0005794', 'GO:0005829', 'GO:0030008', 'GO:0005737', 'GO:1990071', 'GO:1990072', 'GO:0005829']}
428977


In [ ]:
with open('../data/GO/all_download.json') as f:
    download_pdb_go = json.load(f)

In [ ]:
download_pdb_go = [pdbid.upper() for pdbid in download_pdb_go]
download_pdb_go = list(set(download_pdb_go))

In [ ]:
import csv
go_tsv = []
for key, value in tqdm(go_annot.items()):
    if key[:4] in download_pdb_go:
        go_tsv.append([key, ','.join(list(set(value['molecular_function']))), ','.join(list(set(value['biological_process']))), ','.join(list(set(value['cellular_component'])))])

  0%|          | 0/428977 [00:00<?, ?it/s]

100%|██████████| 428977/428977 [27:13<00:00, 262.57it/s]


In [ ]:
go_tsv[0]

['101M-A', 'GO:0005344,GO:0046872', 'GO:0071588,GO:0015671', '']

In [ ]:
mf  = []
bp = []
cc = []
for key, value in tqdm(go_annot.items()):
    if key[:4] in download_pdb_go:
        mf.extend(value['molecular_function'])
        bp.extend(value['biological_process'])
        cc.extend(value['cellular_component'])
mf = list(set(mf))
bp = list(set(bp))
cc = list(set(cc))

  0%|          | 0/428977 [00:00<?, ?it/s]

100%|██████████| 428977/428977 [26:40<00:00, 267.97it/s]


In [ ]:
print(len(mf))
print(len(bp))
print(len(cc))

NameError: name 'mf' is not defined

In [ ]:
mf_name = [go_name_dic[go_mf] for go_mf in mf]
bp_name = [go_name_dic[go_bp] for go_bp in bp]
cc_name = [go_name_dic[go_cc] for go_cc in cc]

In [ ]:
type_list = ['molecular_function', 'biological_process', 'cellular_component']
go_type_list = {
    'molecular_function': [mf, mf_name],
    'biological_process': [bp, bp_name],
    'cellular_component': [cc, cc_name]
}
annot = ""
for t in type_list:
    annot += '### GO-terms ({})'.format(t) +'\n'
    annot += ('\t'.join(go_type_list[t][0]) + '\n')
    annot += '### GO-names ({})'.format(t) +'\n'
    annot += ('\t'.join(go_type_list[t][1]) + '\n')

In [ ]:
with open('../data/GO/nrPDB-GO_annot.tsv', 'w') as f:
    f.writelines(annot)
    for row in go_tsv:
        f.writelines('\t'.join(row) + '\n')

In [ ]:
with open('../data/GO/nrPDB-GO_annot.tsv', 'w') as f:
    go_lines = f.readlines()
for line in go_lines[13:]:
    

In [25]:
for row in go_tsv:
    if '7NS4' in row[0]:
        print(row)

['7NS4-B', 'GO:0005515,GO:0061630,GO:0004842,GO:0008270,GO:0046872,GO:0016740', 'GO:0043161,GO:0006511,GO:0045721,GO:0016567', 'GO:0034657,GO:0005737,GO:0005829,GO:0005777,GO:0005634']
['7NS4-I', 'GO:0061630,GO:0004842,GO:0046872,GO:0016740', 'GO:0043161,GO:0043066,GO:0045721,GO:0016567', 'GO:0034657,GO:0005634,GO:0005737']


In [32]:
go_uniprot_dic = {}
for row in tqdm(pdb_chain_go_rows):
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    if pdb_chain not in go_uniprot_dic.keys():
        go_uniprot_dic[pdb_chain] = []
    go_uniprot_dic[pdb_chain].append(row['uniprot'])
for k, v in tqdm(go_uniprot_dic.items()):
    go_uniprot_dic[k] = list(set(v))

  1%|          | 46719/8650440 [00:00<00:18, 467148.12it/s]

100%|██████████| 397040/397040 [00:02<00:00, 152114.80it/s]


In [33]:
go_uniprots = {}
for go in go_tsv:
    pdbid = go[0]
    go_uniprots[pdbid] = go_uniprot_dic[pdbid]

In [34]:
with open('../output_info/go_uniprots.json', 'w') as f:
    json.dump(go_uniprots, f)

In [36]:
pdb_chain_go_rows[0]

{'PDB': '101m',
 'CHAIN': 'A',
 'uniprot': 'P02185',
 'GO_ID': 'GO:0046872',
 'EVIDENCE': 'IEA'}

In [45]:
pdb_chain_uniprots = {}
for row in pdb_chain_go_rows:
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    if pdb_chain in pdb_chain_uniprots.keys():
        pdb_chain_uniprots[pdb_chain][row['CHAIN']] = row['uniprot']
    else:
        pdb_chain_uniprots[pdb_chain] = {}
        pdb_chain_uniprots[pdb_chain][row['CHAIN']] = row['uniprot']

In [46]:
pdb_chain_uniprots['7V2E-B']

{'b': 'Q29259'}

In [37]:
with open('../output_info/uniprot_dict_all_origin.json') as f:
    uniprots_all = json.load(f)

In [40]:
with open('../datasets/MultiTask_go/test.txt') as f:
    test_list  = f.readlines()
with open('../datasets/MultiTask_go/val.txt') as f:
    val_list  = f.readlines()
with open('../datasets/MultiTask_go/train.txt') as f:
    train_list  = f.readlines()
with open('../datasets/MultiTask_go/train_all.txt') as f:
    train_all_list  = f.readlines()

In [47]:
all_list = test_list + val_list + train_list + train_all_list
for i in all_list:
    i = i[:-1]
    if i not in uniprots_all.keys():
        uniprots_all[i] = pdb_chain_uniprots[i]

In [48]:
with open('../output_info/uniprot_dict_all_go.json', 'w') as f:
    json.dump(uniprots_all, f)

In [ ]:
test_go = '''155C-A	GO:0009055,GO:0046906,GO:0020037	GO:0006091,GO:0022900	
'''

In [ ]:
# test_go = test_go.split('\t')
test_go

In [ ]:
test_go[3].find('48208')

In [ ]:
nr_lines = None
with open('../data/nrPDB-GO_annot.tsv') as f:
    nr_lines = f.readlines()

In [ ]:
go_term_dic = {'molecular_function': [], 'biological_process': [], 'cellular_component': []}
flag = 0
for idx, line in enumerate(nr_lines):
    if line.find('### GO-terms') == 0:
        gos = nr_lines[idx + 1].split('\t')
        gos = [go_id.strip() for go_id in gos]
        if flag == 0:
            go_term_dic['molecular_function'] = gos
        elif flag == 1:
            go_term_dic['biological_process'] = gos
        elif flag == 2:
            go_term_dic['cellular_component'] = gos
        flag += 1

In [ ]:
go_terms = {'molecular_function': [], 'biological_process': [], 'cellular_component': []}
for i in range(13, len(nr_lines)):
    GO_TERM = nr_lines[i][:-1].split('\t')
    if GO_TERM[1]!= '':
        go_terms['molecular_function'].extend(GO_TERM[1].split(','))
    if GO_TERM[2]!= '':
        go_terms['biological_process'].extend(GO_TERM[2].split(','))
    if GO_TERM[3]!= '':
        go_terms['cellular_component'].extend(GO_TERM[3].split(','))

In [ ]:
go_terms['molecular_function'] = list(set(go_terms['molecular_function']))
go_terms['biological_process'] = list(set(go_terms['biological_process']))
go_terms['cellular_component'] = list(set(go_terms['cellular_component']))

In [ ]:
if go_term_dic['cellular_component'].sort() == go_terms['cellular_component'].sort():
    print('true')